<a href="https://colab.research.google.com/github/Alerza/nlp_task/blob/main/Get_fasttext_and_llama_embeding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install fasttext
!pip install transformers

In [59]:
import pandas as pd
llama_vocab = pd.read_csv('/content/drive/MyDrive/DataSet/vocab7b.csv')
llama_vocab.rename(columns = {'Unnamed: 0':'llamaids'}, inplace = True)

In [60]:
llama_vocab

,llamaids,token
0,0,<unk>
1,1,<s>
2,2,</s>
3,3,<0x00>
4,4,<0x01>
...,...,...
31995,31995,黃
31996,31996,왕
31997,31997,收
31998,31998,弘


In [51]:
def remve_under(token):

    cleantokne = str(token).replace('_','')

    return cleantokne

#sample['tokenizer'] = sample['token'].apply(tokne2tokenize)
llama_vocab['clean'] = llama_vocab['token'].apply(remve_under)
llama_vocab

,llamaids,token,clean
0,0,<unk>,<unk>
1,1,<s>,<s>
2,2,</s>,</s>
3,3,<0x00>,<0x00>
4,4,<0x01>,<0x01>
...,...,...,...
31995,31995,黃,黃
31996,31996,왕,왕
31997,31997,收,收
31998,31998,弘,弘


In [5]:
import fasttext
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id="facebook/fasttext-fa-vectors", filename="model.bin")
model_fa = fasttext.load_model(model_path)

In [6]:
model_path = hf_hub_download(repo_id="facebook/fasttext-en-vectors", filename="model.bin")
model_en = fasttext.load_model(model_path)

In [65]:
words_fa = model_fa.get_words()
words_fa_df = pd.DataFrame(words_fa, columns=['token'])
words_fa_df['ftindex'] = words_fa_df.index
words_fa_df = words_fa_df[['ftindex','token']]

words_en = model_en.get_words()
words_en_df = pd.DataFrame(words_en, columns=['token'])
words_en_df['ftindex'] = words_en_df.index
words_en_df = words_en_df[['ftindex','token']]

In [70]:
vocab_en_merge = pd.merge(llama_vocab,words_en_df, how='inner', on='token')
vocab_en_merge = vocab_en_merge[['llamaids','ftindex','token']]
vocab_en_merge

,llamaids,ftindex,token
0,2,7,</s>
1,261,10712,er
2,262,8,in
3,264,1931,en
4,265,17,on
...,...,...,...
10302,31979,1414023,才
10303,31981,674572,止
10304,31983,1045506,ɯ
10305,31987,1476715,역


In [45]:
import torch
llamaemmbding = torch.load('/content/drive/MyDrive/DataSet/embeding_tensor_7b_llama.pt')

In [77]:
import numpy as np
model = model_en


def tokne2llamaEMBD(llamaids):
    llamaEMBD= llamaemmbding[llamaids].detach().numpy()
    return   llamaEMBD

def tokne2fasttextEMBD(ftindex):
    fasttextEMBD= model.get_input_vector(ftindex)
    return   fasttextEMBD


vocab_en_merge[ 'llamaEMBD'] = vocab_en_merge['llamaids'].apply(tokne2llamaEMBD)

vocab_en_merge[ 'fasttextEMBD'] = vocab_en_merge['ftindex'].apply(tokne2fasttextEMBD)
vocab_en_merge

,llamaids,ftindex,token,fasttextEMBD,llamaEMBD
0,2,7,</s>,"[0.073061086, -0.24302974, -0.035331346, -0.36...","[0.001743, 0.001763, -0.014465, -0.01499, 0.00..."
1,261,10712,er,"[-0.019191982, -0.549969, -0.46333933, -0.0955...","[-0.002476, 0.0135, 0.01287, 0.01364, -0.01028..."
2,262,8,in,"[-0.014047037, -0.25217462, 0.07150193, -0.024...","[-0.0119, -0.02966, 0.02472, -0.00782, 0.01378..."
3,264,1931,en,"[0.21992187, -0.6871976, -1.1512233, 0.1031608...","[-0.001209, 0.0001951, -0.02249, 0.01139, -0.0..."
4,265,17,on,"[-0.056821056, 0.5490306, 0.13121544, 0.182454...","[0.005417, -0.02298, 0.02185, 0.009415, -0.020..."
...,...,...,...,...,...
10302,31979,1414023,才,"[-0.27602875, -0.1776536, -0.14800642, -0.2711...","[-0.0308, 0.02261, 0.004223, 0.02196, 0.01126,..."
10303,31981,674572,止,"[0.02189009, -0.18930373, 0.30840504, -0.16206...","[-0.003283, -0.01025, -0.001086, -0.01436, -0...."
10304,31983,1045506,ɯ,"[0.15266347, -0.49368674, -0.16927618, -0.5155...","[-0.02205, 0.001206, -0.001842, -0.001197, -0...."
10305,31987,1476715,역,"[0.1412431, -0.19179188, -0.18136221, -0.39357...","[-0.01988, 0.0133, -0.0001901, -0.011185, 0.02..."
